In [3]:
import pandas as pd

books=pd.read_csv("books_with_categories.csv")

In [4]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = 0)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [5]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [6]:
# Run it over the entire description
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548405289649963},
  {'label': 'neutral', 'score': 0.1698521375656128},
  {'label': 'sadness', 'score': 0.11640947312116623},
  {'label': 'surprise', 'score': 0.020700624212622643},
  {'label': 'disgust', 'score': 0.019100714474916458},
  {'label': 'joy', 'score': 0.015161413699388504},
  {'label': 'anger', 'score': 0.003935149405151606}]]

In [7]:
# Based on individual sentences
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [8]:
# Sort based on label, instead of score
sorted(predictions[0], key=lambda x: x["label"])
predictions

[[{'label': 'surprise', 'score': 0.7296021580696106},
  {'label': 'neutral', 'score': 0.1403859257698059},
  {'label': 'fear', 'score': 0.06816230714321136},
  {'label': 'joy', 'score': 0.047942616045475006},
  {'label': 'anger', 'score': 0.009156371466815472},
  {'label': 'disgust', 'score': 0.0026284768246114254},
  {'label': 'sadness', 'score': 0.002122163772583008}],
 [{'label': 'neutral', 'score': 0.4493703246116638},
  {'label': 'disgust', 'score': 0.27359163761138916},
  {'label': 'joy', 'score': 0.10908312350511551},
  {'label': 'sadness', 'score': 0.09362749755382538},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.026970164850354195},
  {'label': 'fear', 'score': 0.006879044696688652}],
 [{'label': 'neutral', 'score': 0.6462153792381287},
  {'label': 'sadness', 'score': 0.24273383617401123},
  {'label': 'disgust', 'score': 0.043422721326351166},
  {'label': 'surprise', 'score': 0.02830054610967636},
  {'label': 'joy', 'score': 0.0142114

In [9]:
import numpy as np

emotion_labels = sorted(["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"])
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [10]:
# Test out on the first 3 books
for i in range(3):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    print(max_scores)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

{'anger': 0.0641336590051651, 'disgust': 0.27359163761138916, 'fear': 0.9281679391860962, 'joy': 0.9327980279922485, 'neutral': 0.6462153792381287, 'sadness': 0.9671575427055359, 'surprise': 0.7296021580696106}
{'anger': 0.6126202940940857, 'disgust': 0.3482847809791565, 'fear': 0.9425276517868042, 'joy': 0.704422116279602, 'neutral': 0.887939453125, 'sadness': 0.11169023811817169, 'surprise': 0.25254619121551514}
{'anger': 0.0641336590051651, 'disgust': 0.10400677472352982, 'fear': 0.9723207950592041, 'joy': 0.7672382593154907, 'neutral': 0.5494766235351562, 'sadness': 0.11169023811817169, 'surprise': 0.07876548171043396}


In [11]:
emotion_scores

{'anger': [0.0641336590051651, 0.6126202940940857, 0.0641336590051651],
 'disgust': [0.27359163761138916, 0.3482847809791565, 0.10400677472352982],
 'fear': [0.9281679391860962, 0.9425276517868042, 0.9723207950592041],
 'joy': [0.9327980279922485, 0.704422116279602, 0.7672382593154907],
 'neutral': [0.6462153792381287, 0.887939453125, 0.5494766235351562],
 'sadness': [0.9671575427055359, 0.11169023811817169, 0.11169023811817169],
 'surprise': [0.7296021580696106, 0.25254619121551514, 0.07876548171043396]}

In [12]:
# For all books
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████████████████████████████████████████████████████████████████████████| 5197/5197 [15:21<00:00,  5.64it/s]


In [13]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [14]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646215,0.967158,0.729602,9780002005883
1,0.612620,0.348285,0.942528,0.704422,0.887939,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351485,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759457,0.951104,0.368111,0.078765,9788185300535


In [15]:
books = pd.merge(books, emotions_df, on = "isbn13")
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646215,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612620,0.348285,0.942528,0.704422,0.887939,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351485,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765


In [16]:
# For each description, get the max value for each emotion
books.to_csv("books_with_emotions.csv", index = False)